# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import pickle

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddasil28\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///./data/DisasterResponseData.db')
df = pd.read_sql('select * from DisasterResponse', engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = text.lower()

    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tok = []
    for tok in tokens:
        clean_tok.append(lemmatizer.lemmatize(tok).strip())

    return clean_tok

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
target_names = Y_test.columns

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
# Calculate the accuracy for each one
for i in range(len(target_names)):
    print('Target category {}: {} '.format(i, target_names[i]))
    print(classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))
    print('Confusion Matrix: \n {} \n\n'.format(confusion_matrix(y_pred[:, i], Y_test.iloc[:, i])))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, y_pred[:, i])))

In [ ]:
#Clear view droped the first column multiclass and multilabeled
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = target_names[1:]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2))
    , 'vect__max_df': (0.5, 0.75, 1.0)
    , 'tfidf__use_idf': (True, False)
    , 'clf__estimator__n_estimators': [50, 100, 200]
    , 'clf__estimator__min_samples_split': [2, 3, 4]
}

pipeline_cv = GridSearchCV(pipeline, param_grid = parameters)

pipeline_cv.fit(X_train, Y_train)

In [ ]:
print(pipeline_cv.best_params_)
print(pipeline_cv.best_score_)
print(pipeline_cvl.best_estimator_)
print(pipeline_cv.grid_scores_)

In [ ]:
#building new model
optimised_model = pipeline_cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
optimised_model.fit(X_train, Y_train)
print('Best Parameters:', pipeline_cv.best_params_)

In [ ]:
y_pred = optimised_model.predict(X_test)
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred]), target_names = target_names[1:]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

parameters_kn = {'tfidf__use_idf':[True, False],
                 'clf__estimator__weights':['uniform','distance'],
                 'clf__estimator__leaf_size':[25,35]
                }
cv = GridSearchCV(pipeline, param_grid = parameters_kn, verbose = 10, n_jobs=-1)

np.random.seed(42)

kn_model = cv.fit(X_train, Y_train)

In [ ]:
print(kn_model.best_params_)
print(cv.best_score_)

### 9. Export your model as a pickle file

In [ ]:
with open('./model/classifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.